# Пример коришћења алата AutoGen


## Увоз потребних пакета


In [ ]:
import os
import json

import requests
from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken
from autogen_core.tools import FunctionTool
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console
from typing import Any, Callable, Set, Dict, List, Optional

## Креирање клијента

У овом примеру, користићемо [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) за приступ LLM-у.

`model` је дефинисан као `gpt-4o-mini`. Пробајте да промените модел на неки други доступан у GitHub Models продавници како бисте видели различите резултате.

Као брз тест, само ћемо покренути једноставан упит - `Који је главни град Француске`.


In [ ]:
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.environ["GITHUB_TOKEN"]),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

## Дефинисање функција

У овом примеру, даћемо агенту приступ алату који је функција са списком доступних дестинација за одмор и њиховом доступношћу.

Можете замислити да би ово била ситуација у којој туристички агент има приступ туристичкој бази података, на пример.

Док пролазите кроз овај пример, слободно покушајте да дефинишете нове функције и алате које агент може позвати.


In [ ]:
from typing import Dict, List, Optional


def vacation_destinations(city: str) -> tuple[str, str]:
    """
    Checks if a specific vacation destination is available
    
    Args:
        city (str): Name of the city to check
        
    Returns:
        tuple: Contains city name and availability status ('Available' or 'Unavailable')
    """
    destinations = {
        "Barcelona": "Available",
        "Tokyo": "Unavailable",
        "Cape Town": "Available",
        "Vancouver": "Available",
        "Dubai": "Unavailable",
    }

    if city in destinations:
        return city, destinations[city]
    else:
        return city, "City not found"

# Example usage:
# city, status = vacation_destinations("Barcelona")
# print(f"How about visiting {city}? It's currently {status} there!")

## Дефинисање алата функције  
Да би агент користио `vacation_destinations` као `FunctionTool`, потребно је да га дефинишемо као такав.  

Такође ћемо пружити опис алата који је користан агенту да идентификује за шта се тај алат користи у односу на задатак који је корисник затражио.  


In [ ]:
get_vacations = FunctionTool(
    vacation_destinations, description="Search for vacation destinations and if they are available or not."
)

## Дефинисање агента

Сада можемо креирати агента у коду испод. Дефинишемо `system_message` како бисмо дали агенту инструкције о томе како да помогне корисницима у проналажењу дестинација за одмор.

Такође постављамо параметар `reflect_on_tool_use` на true. Ово омогућава коришћење LLM-а да узме одговор из позива алата и пошаље одговор користећи природни језик.

Можете поставити овај параметар на false да бисте видели разлику.


In [ ]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[get_vacations],
    system_message="You are a travel agent that helps users find vacation destinations.",
    reflect_on_tool_use=True,
)

## Покретање агента

Сада можемо покренути агента са почетном поруком корисника који тражи путовање у Токио.

Можете променити ову дестинацију града да бисте видели како агент реагује на доступност града.


In [ ]:
async def assistant_run() -> None:
    response = await agent.on_messages(
        [TextMessage(content="I would like to take a trip to Tokyo", source="user")],
        cancellation_token=CancellationToken(),
    )
    print(response.inner_messages)
    print(response.chat_message)


# Use asyncio.run(assistant_run()) when running in a script.
await assistant_run()


---

**Одрицање од одговорности**:  
Овај документ је преведен коришћењем услуге за превођење помоћу вештачке интелигенције [Co-op Translator](https://github.com/Azure/co-op-translator). Иако се трудимо да обезбедимо тачност, молимо вас да имате у виду да аутоматски преводи могу садржати грешке или нетачности. Оригинални документ на његовом изворном језику треба сматрати ауторитативним извором. За критичне информације препоручује се професионални превод од стране људи. Не преузимамо одговорност за било каква погрешна тумачења или неспоразуме који могу настати услед коришћења овог превода.
